In [227]:
import numpy as np
from numpy.random import random
import torch
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal as MVN
import pdb
from vb_yazid import *
import scipy.io
import os
import time
from scipy.stats import multivariate_normal
from tqdm.notebook import trange, tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [228]:
a = np.random.randn(10,2)

b = np.ones([20]).astype(int)


In [229]:
dim_x=1 # dimension etat_cache
dim_y=1 # dimension observation
K=2  #nombre de sauts
T=1000 #nombre d'observations 

#Définition du modèle (F,H,Q,R,...)

T_e=1
virage=6*np.pi/180
omega=np.array([0,virage,virage])
#sigma_Q = np.array([3,10,10,10,10,10,10])
sigma_Q = 10*np.array([1,1])
sigma_R = 1
#a=np.array([1,-0.9,0.9,-0.8,0.8,-0.7,0.7])
a=np.array([0.95,-0.95])

prob=0.8
#mat_trans=np.array([[0.8,0.1,0.1],[0.1, 0.8,0.1],[0.1,0.1,0.8]]) #p(r_t=j|r_{t-1}=i)=mat[i,j]
mat_trans1=np.diag(prob*np.ones(K))
mat_trans2=(1-prob)/(K-1)*np.ones([K,K])
np.fill_diagonal(mat_trans2,0)
mat_trans=mat_trans1+mat_trans2
#print(mat_trans)
H_def=np.identity(dim_y) # si dim_x=4 et dim_y=4
#H_def=np.array([[1.0,0,0,0],[0,0,1.0,0]])
H=np.tile(H_def,(K,1,1))#ne dépend pas de r
R=sigma_R*np.identity(dim_y) #ne dépend pas de r
F=np.zeros((K,dim_x,dim_x))
Q=np.zeros((K,dim_x,dim_x))

In [192]:
import torch
from torch.distributions import MultivariateNormal, Normal

In [208]:
mvn = MultivariateNormal(torch.zeros(50,1), torch.tensor(var_particules).float())

mvn.log_prob(torch.ones(50,1))

tensor([-2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850,
        -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850,
        -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850,
        -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850,
        -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850,
        -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850, -2.8850,
        -2.8850, -2.8850])

In [212]:
var_particules.shape

(50, 1, 1)

In [279]:
import torch
from torch.distributions import MultivariateNormal, Normal

def filtre_kalman_pmc(F,H,Q,R,B,sigma,moy,var,z,z_prec,dim_x,dim_y):
    
    pred_moy= B@np.concatenate((moy,z_prec))
    pred_cov=B[0:dim_x+dim_y,0:dim_x]@var@B[0:dim_x+dim_y,0:dim_x].transpose()+sigma
    moy=pred_moy[0:dim_x]+pred_cov[0:dim_x,dim_x:dim_x+dim_y]@np.linalg.inv(pred_cov[dim_x:dim_x+dim_y,dim_x:dim_x+dim_y])@(z-pred_moy[dim_x:dim_x+dim_y])
    var=pred_cov[0:dim_x,0:dim_x]-pred_cov[0:dim_x,dim_x:dim_x+dim_y]@np.linalg.inv(pred_cov[dim_x:dim_x+dim_y,dim_x:dim_x+dim_y])@pred_cov[0:dim_x,dim_x:dim_x+dim_y].transpose()
    
    return moy, var


def multinomial_resample(weights):
    
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1.  # avoid round-off errors: ensures sum is exactly one
    return np.searchsorted(cumulative_sum, random(len(weights)))

def parallel_choice(probs):
    unif = np.random.uniform(0,1,probs.shape)
    probs = probs.cumsum(1)

    return (unif < probs).argmax(axis=1)

def FiltreParticulaire_bootstrap_parallel(particules,poids,moy,var,y,Q,R,F,H,mat_trans):
    
    dim_x=np.size(moy,1)
    dim_y=np.size(y,0)
    N=np.size(particules,0)
    K=np.size(mat_trans,0)
    
    particules = parallel_choice(mat_trans[particules])
    
    y_tilde = y - H[particules] @ F[particules] @ moy[..., None]
    var_pred = F[particules] @ var @ F[particules].transpose((0,2,1)) + Q[particules]
    S = H[particules] @ var_pred @ H[particules].transpose((0,2,1)) + R
    K_k = var_pred @ H[particules].transpose((0,2,1)) @ np.linalg.inv(S)
    var = (np.eye(dim_x) - K_k @ H[particules]) @ var_pred 
    moy = F[particules] @ moy[..., None] + K_k @ y_tilde

    mvn = MultivariateNormal(torch.zeros(N,1), torch.tensor(S).float())
    poids = poids * mvn.log_prob(torch.tensor(y_tilde).squeeze(-1).float()).exp().numpy() 
    
    poids = poids/np.sum(poids)
    #print(moy)
    estime = (poids * moy).sum()
    poids = poids.reshape(-1,)
    moy = moy.reshape(-1,1)
    neff = 1/np.sum(poids**2)
    #print(moy)

    if neff/N <0.3:
        #print(poids)
        index = multinomial_resample(poids)
         
        poids=np.ones(N)/N
        particules=particules[index,]
        moy=moy[index,]
        var=var[index,]
        
    return estime,particules,poids,moy,var

def FiltreParticulaire_bootstrap(particules,poids,moy,var,y,Q,R,F,H,mat_trans):
    
    dim_x=np.size(moy,1)
    dim_y=np.size(y,0)
    N=np.size(particules,0)
    K=np.size(mat_trans,0)

    for j in range(0,N) :
        
        #print(mat_trans[particules[j],])
        particules[j]=np.random.choice(range(0,K),1,p=mat_trans[particules[j],])

        y_tilde=y-H[particules[j],]@F[particules[j],]@moy[j,]
        
        var_pred=F[particules[j],]@var[j,]@F[particules[j],].transpose()+Q[particules[j],]
        
        S=H[particules[j],]@var_pred@H[particules[j],].transpose()+ R
        
       
        #print(multivariate_normal(y_tilde,cov=S))
        poids[j]=poids[j]*multivariate_normal.pdf(y_tilde,cov=S)
       
        K_k=var_pred@H[particules[j],].transpose()@np.linalg.inv(S)
         
        var[j,]=(np.eye(dim_x)-K_k@H[particules[j],])@var_pred
        
        moy[j,]=F[particules[j],]@moy[j,]+K_k@y_tilde
        
    poids=poids/np.sum(poids)

    
    estime=poids@moy
    moment_deux=poids@(np.sum(var,axis=(1,2))+np.sum(moy**2,axis=1))
    neff=1/np.sum(poids**2)
    #print(moy)

    if neff/N <0.3:
        #print(poids)
        index = multinomial_resample(poids)
         
        poids=np.ones(N)/N
        particules=particules[index,]
        moy=moy[index,]
        var=var[index,]
        
    return(estime,particules,poids,moy,var)


a, *b = FiltreParticulaire_bootstrap_parallel(particules,poids,moy_particules,var_particules,y,Q,R,F,H,mat_trans)

[[[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]]
[[[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]] [[[2.94008861]]

 [[2.94008861]]

 [[2.94008861]]

 [[2.94008861]]

 [[2.94008861]]

 [[2.94008861]]

In [277]:
N = 50

loi_saut_init=(1-prob)/(K-1)*np.ones(K)
loi_saut_init[0]=prob
x_init=np.array([5])
var_particules=np.tile(50*np.eye(dim_x),(N,1,1))
moy_particules=np.tile(1.0*x_init,(N,1))
particules=np.random.choice(range(0,K),N,p=loi_saut_init)
poids=np.ones(N)/N
moy_fp=np.zeros((T,dim_x))
moy_fp[0,]=x_init

var_particules_var=np.tile(50*np.eye(dim_x),(N,1,1))
moy_particules_var=np.tile(1.0*x_init,(N,1))
particules_var=np.random.choice(range(0,K),N,p=loi_saut_init)
poids_var=np.ones(N)/N
moy_fp_var=np.zeros((T,dim_x))
moy_fp_var[0,]=x_init

for t in range(20):
    y = observations[t]
    [moy_fp[t,],particules,poids,moy_particules,var_particules]=FiltreParticulaire_bootstrap(particules,poids,moy_particules,var_particules,y,Q,R,F,H,mat_trans)
    [moy_fp[t,],particules,poids,moy_particules,var_particules]=FiltreParticulaire_bootstrap_parallel(particules,poids,moy_particules,var_particules,y,Q,R,F,H,mat_trans)

[[[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]

 [[5.]]] [[[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]

 [[0.]]]
[[[11.50505052]]

 [[11.50505052]]

 [[11.50505052]]

 [[11.50505052]]

 [[11.50505052]]

 [[11.5050

<ipython-input-276-35c33672e2f5>:45: RuntimeWarning: invalid value encountered in true_divide
  poids = poids/np.sum(poids)
